In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

from functions import import_imagedata, label_oh_tf, ImageProcessor
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models, transforms
import torch.optim as optim
from torch.optim import lr_scheduler

from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from torchvision.models import vgg16
from torchvision.models import resnet101

import cv2

from torch.utils.data import DataLoader
import wandb
from fns4wandb import train_log, build_optimizer
from copy import deepcopy
from tqdm import tqdm

import random
import pickle
from fns4wandb import set_lossfn

In [ ]:
device = "cuda:1" if torch.cuda.is_available() else "cpu"

In [ ]:
model_resnet = resnet101(weights="IMAGENET1K_V1")#.eval
print(model_resnet)

In [ ]:
# Step 1: Initialize model with the best available weights
#model_resnet_featlayers = resnet101(weights="IMAGENET1K_V1").features#.to(device)
#model_resnet_featlayers.eval()
newmodel = torch.nn.Sequential(*(list(model_resnet.children())[:-1]))
newmodel=newmodel.to(device)
newmodel = newmodel.eval()
#print(newmodel)

In [ ]:
# Step 2: Initialize the inference transforms
# This does some preprocessing behind the scenes,

# 1) Resized of the input to resize_size=[256];
# 2) Followed by a central crop of crop_size=[224];

# vgg16 and resnet accept input image size of 224×224



#print(preprocess)

In [ ]:
# create MLP, linear classification, function

class Three_Lay_MLP(nn.Module):
    def __init__(self):
        super(Three_Lay_MLP, self).__init__()
        
        self.lins = nn.Sequential(
            nn.Linear(2048, 100), #1024x7 and 1024x100. 7, 1024
            nn.ReLU(),
            nn.Linear(100,100),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(100,11),
            nn.Linear(11,11),
            nn.Softmax(),
        )
    def forward(self, x):
        x = self.lins(x)
        return x
        
#lin    conv output -> 100
#relu    ()
#lin    100 -> 100
#relu    ()
#do     ~0.5
#lin    100 -> 11
#softmax ()

In [ ]:
def metrics(labels, predictions): #TypeError: Singleton array tensor(3) cannot be considered a valid collection.
    #print('l  ',label) #len(11)

    labels = [np.argmax(label.cpu().detach().numpy()) for label in labels]
    predictions = [np.argmax(prediction.cpu().detach().numpy()) for prediction in predictions]


    avg_f1_score = f1_score(labels, predictions, average='macro')

    acc = accuracy_score(labels, predictions)
    
    return avg_f1_score, acc

In [ ]:
"""from sklearn.preprocessing import StandardScaler

# creating a function for normalizing the feature vector
# using a standard scaling (results in mean=0 and standard deviation=1)
def scale_feats(feat_vec):
  # Scaling the features to the same range of values
  scaler = StandardScaler()
  scaler.fit(feat_vec)
  scaled_feat_vec = scaler.transform(feat_vec)
  print("\n A peek at the scaled dataset features: \n"+str(scaled_feat_vec))

  return scaled_feat_vec

# normalize the feature vector
scaled_feats_28 = scale_feats(feats_28)"""

In [ ]:
# Temi code. week 6. notebook used vgg16 pretrained features, and trained a MLP for classification of beans
# this is the method I am following 

IP = ImageProcessor(device='cpu')

# Temi func. get features from passing through images into vgg16 conv/ feature layers.
# creating a function to get features learnt in the pretrained VGG16
# extracting these features for our own list of images
def get_img_feats(img_path): 
    #print(img_path)
    img = cv2.imread(img_path) #
    #print(img)
    img = IP.blank_padding(img, (224,224))
    img = IP.to_tensor(img).to(device)
    #print('image shape post read and pad: \n ',img.shape)
    # Step 3: Apply inference preprocessing transforms
    #img = preprocess(img).unsqueeze(0) # preprocess is the model with weights
    #img = preprocess(img)#.unsqueeze(0)

    # Step 4: Use the model and print the predicted category
    #print('get_img_feats img', img.shape) 
    #auto_feats = auto_feats.detach().numpy() 
    #Tensor.cpu() to copy the tensor to host memory first
    auto_feats = newmodel(img).squeeze(0)
    
    
    #print('get_img_feats auto_feats', auto_feats.shape)
    
    # AttributeError: 'builtin_function_or_method' object has no attribute 'detach' --- current error
    #AttributeError: 'numpy.ndarray' object has no attribute 'to'
    auto_feats = auto_feats.cpu().detach().numpy()
    
    #print('get_img_feats auto_feats detached', auto_feats.shape)
    auto_feats = np.mean(auto_feats, axis=1,keepdims=False) #, keepdims=False
    #print('get_img_feats auto_feats np mean', auto_feats.shape)
    #print('get_img_feats autofeats', auto_feats)
    #print('auto_feat: \n ', auto_feats)
    auto_feats= torch.tensor(auto_feats, dtype=torch.float32)
    return auto_feats.to(device)


# class to manage data and turn imgs into vgg16 features

"""class AutoFeature(): # file_path, number
    def __init__(self, img_files, labels):
        self.img_files = img_files
        self.label = label_oh_tf(labels, device='cpu', num_classes=11)
        
        #self.labels=[label_oh_tf(label, device='cpu', num_classes=11) for label in labels]
        #self.labels= [int(label) for label in labels]
        #self.labels = torch.tensor(labels, dtype=torch.long) # tensofy the labels
        #self.labels = label_oh_tf(labels, device, 11)
    #def __len__(self):
    #    return len(self.labels)
    def __getitem__(self):
        #given an index, will return items within that index range
        # extracting features using pretrained model
        feats = get_img_feats(self.img_files) #[idx]
        
        feats_tensor = torch.tensor(feats, dtype=torch.float32)
        #print(type(self.labels))
        #print(self.labels[3])
        return feats_tensor, self.label #[idx]"""

In [ ]:
# get len of 

file_path = r'/its/home/nn268/optics/AugmentedDS_IDSW/'
random_seed =1
img_len = len(os.listdir(file_path))

ids = np.arange(0, img_len)
#print(ids[4])


train_ids, test_ids = train_test_split(ids, test_size=0.2, train_size=0.8,
                                 random_state=random_seed, shuffle=True)
train_ids, val_ids = train_test_split(train_ids, test_size=0.1, train_size=0.8,
                                 random_state=random_seed, shuffle=True)


In [ ]:
wandb.login()


In [ ]:

title = f'IDSW_RESNET_MLP_hp_80_112023'
save_dict = {'Run' : title,
            'Current_Epoch': 0}

save_location = r'pickles/'

In [108]:
run_title = "IDSW_RESNET_MLP_hp_80_112023"

config = {
    'method': 'random',
    'metric':{
        'goal': 'minimize',
        'name': 'val_loss'},
    'parameters': {
        #'dropout':{
        #    'values': [0.5, 0.4, 0.3]
        #},
        'epochs':{
            'value': 100
        },
        'first_lin_lay':{
            'values':[248832]
        },
        'optimizer': {
            'values': ['adam']
        },
            'learning_rate': {
                # a flat distribution between 0 and 0.1
                'distribution': 'log_uniform_values',
                'min': 1e-7,
                'max': 1e-2
            },
        'loss_fn': {
            'values': ['CrossEntropy', 'MSE'] #'MSE', 
        },
        'data_set':{
            'values':['Augmented']
        },
            'scheduler': {
            'values': [0.2, 0.3, 0.4, 0.6]
        },
        'ks': {
            'values': [(3,5)]
        },
        'channels':{
            'values': [3]
        },
        'num_classes': {
            'values': [11]
        },
        'model_name' : {'values': ['resnet_mlp']},
        'channels' : {'values': [3]},
        'image_path': {
            'values': [r'/its/home/nn268/optics/AugmentedDS_IDSW/']
        }
        }
    }

sweep_id = wandb.sweep(config, project=f"{run_title}")

Create sweep with ID: edhshryy
Sweep URL: https://wandb.ai/antvis/IDSW_RESNET_MLP_hp_80_112023/sweeps/edhshryy


In [109]:
from torch.utils.data import DataLoader
import wandb
from fns4wandb import train_log, build_optimizer
from copy import deepcopy
from tqdm import tqdm

# split data intro train, val, test

# instancing the autofeature class
# creating an object for train, val test

#train_set = AutoFeature(x[train_ids], y[train_ids])
#val_set = AutoFeature(x[val_ids], y[val_ids])
#test_set = AutoFeature(x[test_ids], y[test_ids])

#print('train set 0: \n',train_set[0])
#print('what is train set?', type(train_set))
#print('train set len', len(train_set))



#x_test, y_test = test_set
#x_test = [i[0] for i in test_set]
#y_test = [i[1] for i in test_set]

#train_dl = DataLoader(train_set, batch_size=1)
#val_dl = DataLoader(val_set, batch_size=1)
#test_dl = DataLoader(test_set, batch_size=1)




"""config = dict(
    epochs= 2, #80, #30, 
    learning_rate =1e-5,
    architecture ='CNN',
    optimizer= 'adam',
    weight_decay= 4e-5,
    ks = 3,
    scheduler=0.2,
    f_lin_lay = 7168 #1024*7 = 7168
)
"""
# pass in ids
# create class instance for single ids
# index that class to get img feature and label FOR THAT ids.
# per epoch



def train_model(model, train_ids, val_ids, config, best_acc=0): #train_dl, val_dl, 
    #wandb.watch(model, loss_fn, log='all', log_freq=10)
    
    loss_fn = set_lossfn(config.loss_fn) # ****
    
    lr = config['learning_rate'] #1e-5 #config.learning_rate
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)#build_optimizer(model, optimizer=torch.optim.Adam(model.parameters(), lr=lr))#config.optimizer, config.learning_rate, config.weight_decay)
    scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=config['scheduler'], last_epoch=-1) #gamma=config.scheduler, last_epoch=-1)
    
    x, y = import_imagedata(file_path)
    #print(x[1].shape)
    
    model.train()
    
    t_loss_list = []
    v_loss_list = []
    t_predict_list = []
    v_predict_list = []
    t_accuracy_list = []
    v_accuracy_list = []
    t_label_list = []
    v_label_list = []
    #labels = []
    
    
    total_epochs = 0
    
    for epoch in tqdm(range(config['epochs'])): #config.epochs)):
        print('Epoch:   ', epoch)
        v_correct = 0
        t_correct = 0
        
        if epoch == 0:
            best_model = deepcopy(model)
        #train_ids = random.shuffle(train_ids)
        #print(type(train_ids))
        print('training...')
        for idx in train_ids: 
            model.train()
            
            x_train = get_img_feats(x[idx])
            tensor = torch.tensor(x_train, dtype=torch.float32)
            tensor = tensor.flatten()

            train_prediction = model.forward(tensor)
            train_label = label_oh_tf(y[idx], device, num_classes=11)
            
            t_loss = loss_fn(train_prediction, train_label)
            if train_prediction.argmax() == train_label.argmax():
                t_correct+=1
   
            t_loss_list.append(t_loss)
            t_predict_list.append(train_prediction)
            
            train_acc = t_correct/len(train_ids)
            t_label_list.append(train_label)
            
            optimizer.zero_grad()
            t_loss.backward()
            
            optimizer.step()
            scheduler.step()
            
            
        print('validation... ')
        for idx in val_ids:
            model.eval()
            
            x_val = get_img_feats(x[idx])
 
            tensor = torch.tensor(x_val, dtype=torch.float32)
            tensor = tensor.flatten()
            y_val = label_oh_tf(y[idx], device, num_classes=11)

            val_prediction = model.forward(tensor)
            #print('val prediction \n     ',val_prediction.argmax())
            v_loss = loss_fn(val_prediction, y_val)
            
            if val_prediction.argmax() == y_val.argmax():
                v_correct +=1
            
            val_acc = (v_correct / len(val_ids))
            v_loss_list.append(v_loss)
            v_predict_list.append(val_prediction)
            #print('val predict list \n       ',v_predict_list)
            v_label_list.append(y_val)
            #print('val label list \n      ',v_label_list)
            
        print('Val Acc:   ',val_acc) 
        
            #print('val accuracy:     ', v_correct/len(val_ids))
            #val_avg_f1_score, val_acc = metrics(y_val, val_prediction)
        if val_acc > best_acc:
            best_acc = val_acc
            best_model = deepcopy(model)
            
            save_dict['Current_Epoch'] += config['epochs']
            save_dict['model.state_dict'] = model.state_dict()
            save_dict['training_samples'] = len(train_ids)
            save_dict['validation_samples'] = len(val_ids)
            save_dict['t_loss_list'] = t_loss_list
            save_dict['v_loss_list'] = v_loss_list
            save_dict['t_predict_list'] = t_predict_list  
            save_dict['v_predict_list'] = v_predict_list
            save_dict['t_accuracy_list'] = t_accuracy_list  #
            save_dict['v_accuracy_list'] = v_accuracy_list 
            save_dict['t_labels'] = t_label_list
            save_dict['v_labels'] = v_label_list
            title = save_dict['Run']
            with open(f'{save_location}{title}.pkl', 'wb+') as f:
                pickle.dump(save_dict, f)
            
            
            print('improvment in metrics. model saved')
        
        if (epoch+1)%2==0:
            print('updating wandb')
            train_log(t_loss, v_loss, epoch)
            wandb.log({'train_accuracy_%': train_acc, 'epoch':epoch})
            wandb.log({'val_accuracy_%': val_acc, 'epoch':epoch})
            
    model = best_model

    return model, save_dict


def pipeline(config): 
    
    loss_list=[]
    #loss_fn = nn.CrossEntropyLoss()
    with wandb.init(project=title, config=config):
        config = wandb.config
        model = Three_Lay_MLP().to(device)

        model, save_dict = train_model(model, train_ids, val_ids, config) #train_dl, val_dl
        
    return model, save_dict

In [ ]:
def tr(config=None):
    with wandb.init(config=config):
        config = wandb.config
        model, save_dict = pipeline(config)

wandb.agent(sweep_id, tr, count=20)

wandb: Agent Starting Run: qwpr6wzw with config:
wandb: 	channels: 3
wandb: 	data_set: Augmented
wandb: 	epochs: 100
wandb: 	first_lin_lay: 248832
wandb: 	image_path: /its/home/nn268/optics/AugmentedDS_IDSW/
wandb: 	ks: [3, 5]
wandb: 	learning_rate: 0.000208212303935189
wandb: 	loss_fn: MSE
wandb: 	model_name: resnet_mlp
wandb: 	num_classes: 11
wandb: 	optimizer: adam
wandb: 	scheduler: 0.3


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


  0%|          | 0/100 [00:00<?, ?it/s]<ipython-input-109-e88fb3493d24>:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor = torch.tensor(x_train, dtype=torch.float32)
/its/home/nn268/.local/lib/python3.8/site-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch:    0
training...
validation... 


<ipython-input-109-e88fb3493d24>:121: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor = torch.tensor(x_val, dtype=torch.float32)


...

Val Acc:    0.09818181818181818
Epoch:    39
training...
validation... 


 40%|████      | 40/100 [32:53<50:13, 50.23s/it]

Val Acc:    0.09818181818181818
updating wandb
Epoch:    40
training...
validation... 


 41%|████      | 41/100 [35:09<1:14:51, 76.13s/it]

Val Acc:    0.09818181818181818
Epoch:    41
training...


In [ ]:
#model, save_dict = pipeline(config) #7,168

In [ ]:
import matplotlib.pyplot as plt

def plot_confusion(predictions:list, actual:list, title:str):
    predict_list = [int(t.argmax()) for t in predictions]
    actual = [int(l.argmax()) for l in actual]

    actual = np.array(actual)
    predict_list = np.array(predict_list)


    #FixedLocator locations (3), usually from a call to set_ticks, does not match the number of labels (11).
    print(f'\n     {title}')
    train_epoch_matrix = confusion_matrix(actual, predict_list, labels= [0,1,2,3,4,5,6,7,8,9,10])
    disp= ConfusionMatrixDisplay(train_epoch_matrix, display_labels=[0,1,2,3,4,5,6,7,8,9,10])
    disp.plot()
    plt.show()

    
#t_predict = predictions[0]
#v_predict = predictions[1]
#t_labels = labels[0]
#v_labels = labels[1]

#print(len(labels))
#for label in labels:
#    print(label[1])
    #t_labels = label[0]
    #v_labels = label[1]
#print(len(t_labels))
#print(len(v_labels))

In [ ]:
print(save_dict.keys())

In [ ]:
t_predict = save_dict['t_predict_list']
t_labels = save_dict['t_labels']

v_predict = save_dict['v_predict_list'] # WHY IS THERE NOTHING IN V OREDICT LIST!
v_labels = save_dict['v_labels']


print(len(v_labels))
print(len(v_predict))

print(len(t_labels))
print(len(t_predict))

In [ ]:
plot_confusion(t_predict, t_labels, 'Train Confusion Matrix'+str(save_dict['Current_Epoch']))
plot_confusion(v_predict, v_labels, 'Validation Confusion Matrix'+str(save_dict['Current_Epoch']))